### Identify Relative Humidity at 850mb over Lakes in NY



##### Step 0: Imports

Import required libraries

In [1]:
from herbie import Herbie
from herbie.toolbox import EasyMap

import numpy as np
import pandas as pd
import xarray as xr

import geopandas as gpd
from shapely.geometry import Point
import fiona  # used for listlayers / FileGDB access

from scipy.interpolate import griddata

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import urllib
from pathlib import Path
import requests

##### Step 1: Datetime Function

Function to allow for parameter-less input and ensure function runs smoothly. Also, localizes to UTC.

In [2]:
#Helper for getting the right date/time
def herbieDateTime(dateTime=None):
    if dateTime is None:
        dt = pd.Timestamp.utcnow().floor("h") - pd.Timedelta(hours=2)
    else:
        dt = pd.Timestamp(dateTime)

    if dt.tzinfo is not None:
        dt = dt.tz_convert("UTC").tz_localize(None)
    return dt

##### Step 2: Fetch dataframes for 850mb Relative humidity.

In [3]:
def rh850(dateTime=None):
    #Convert time entered to UTC
    dt = herbieDateTime(dateTime)
    #HRRR PRS Herbie variable
    H = Herbie(
        dt,
        model="hrrr",
        product="prs",      #3D pressure level fields    
        fxx=0,                  
        save_dir=Path("herbie_cache"),
        overwrite=False
    )

    #GRIB2 subset only RH at 850 mb 
    pat = r":RH:850 mb:"
    H.download(pat, verbose=False)

    #Read GRIB2 in xarray.Dataset
    ds = H.xarray(pat, remove_grib=True)

    return ds


In [4]:
rh850("2025-1-1 0:00")

✅ Found ┊ model=hrrr ┊ product=prs ┊ 2025-Jan-01 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


/opt/miniconda3/envs/herbie-hrrr/lib/python3.11/site-packages/herbie/core.py:1280: UserWarning: Will not remove GRIB file because it previously existed.
  warnings.warn("Will not remove GRIB file because it previously existed.")


<xarray.Dataset> Size: 38MB
Dimensions:              (y: 1059, x: 1799)
Coordinates:
    latitude             (y, x) float64 15MB ...
    longitude            (y, x) float64 15MB ...
    time                 datetime64[ns] 8B 2025-01-01
    step                 timedelta64[ns] 8B 00:00:00
    isobaricInhPa        float64 8B 850.0
    valid_time           datetime64[ns] 8B ...
    gribfile_projection  object 8B None
Dimensions without coordinates: y, x
Data variables:
    r                    (y, x) float32 8MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    model:                   hrrr
    product:                 prs
    description:             High-Resolution Rapid Refresh - CONUS
    remote_grib:             https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr....
    local_grib:              herbie_cache/hrrr/20250101/subset_6aef2a8b__hrrr...
    search:                  :RH:850 mb:

##### Step 3: Fetch the lake polygon

This process is the exact same as the lake temperature notebook. Use USGS NHD, download the GeoJSON file.

In [5]:
#Fetch a lake polygon from USGS NHD and save as GeoJSON
#Lake_name should be provided as a string (e.g. "Cayuga Lake")

def fetch_lake_geojson(lake_name="Cayuga Lake"):
    overwrite=False
    timeout=60
    out_dir = Path("finger_lakes_geojson")
    out_dir.mkdir(parents=True, exist_ok=True)

    fname = lake_name.lower().replace(" ", "_") + ".geojson"
    out_path = out_dir / fname

    if out_path.exists() and not overwrite:
        return out_path

    base = "https://hydro.nationalmap.gov/arcgis/rest/services/nhd/MapServer/12/query"

    where = f"GNIS_NAME = '{lake_name}' AND FTYPE = 390"
    params = {
        "where": where,
        "outFields": "GNIS_NAME,FTYPE",
        "returnGeometry": "true",
        "f": "geojson",
        "outSR": "4326",
    }

    url = base + "?" + urllib.parse.urlencode(params)
    r = requests.get(url, timeout=timeout)
    r.raise_for_status()
    geojson = r.json()

    features = geojson.get("features", [])
    if len(features) == 0:
        raise ValueError(f"No features returned for lake '{lake_name}'")
    if len(features) > 1:
        raise ValueError(
            f"Multiple features returned for lake '{lake_name}' "
            f"({len(features)} features). Refine query."
        )
    out_path.write_bytes(r.content)

    print(
        f"Saved {lake_name}: "
        f"{len(r.content):,} bytes → {out_path}"
    )

    return out_path

##### Step 4: Get the points for the grid within the lake

This step is also identical to the lake temperature notebook. Overlay the rectangular grid, pick the points within the GeoFrame.

In [6]:
def getLakeGrid(lake_name="Cayuga Lake", spacing=0.02):
    lake_geoJson = gpd.read_file(fetch_lake_geojson(lake_name))
    lakeGeom = lake_geoJson.geometry.iloc[0]
    minLon, minLat, maxLon, maxLat = lakeGeom.bounds #Define bounds for the rectangle surrounding the lake

    candidate_lons = np.arange(minLon, maxLon, spacing)
    candidate_lats = np.arange(minLat, maxLat, spacing)

    interior_points = []

    for lat in candidate_lats:
        for lon in candidate_lons:
            p = Point(lon, lat)
            if lakeGeom.contains(p):
                interior_points.append((lat, lon))
    return interior_points

##### Step 5: Find the Relative Humidity 


In [7]:

def rh850Grid(lake_name="Cayuga Lake", dateTime=None, spacing=0.02):
    dateTime = herbieDateTime(dateTime)

    interior_points = getLakeGrid(lake_name)
    inLats = [p[0] for p in interior_points]
    inLons = [p[1] for p in interior_points]
    #Specify the points to be passed to Herbie's pick_points() function
    points = pd.DataFrame(
        {
            "latitude" : inLats,
            "longitude" : inLons,
        }
    )

    ds850 = rh850(dateTime)
    sampled = ds850.herbie.pick_points(points)

    rh_vals= sampled["r"].values

    projected = gpd.GeoDataFrame(
        {
            "lake": lake_name,
            "timestamp": pd.Timestamp(dateTime),
            "latitude": inLats,
            "longitude": inLons,
            "rh850": rh_vals,
        },
        geometry=gpd.points_from_xy(inLons, inLats),
        crs="EPSG:4326",
    )
    return projected

##### Step 6: Take the average of Relative Humanity 850mb return output as a Pandas dataframe


In [8]:
def avgRH850(lake_name="Cayuga Lake", dateTime = None, spacing = 0.02):
    dateTime = herbieDateTime(dateTime)
    grid = rh850Grid(lake_name, dateTime, spacing)
    if grid.empty:
        return pd.DataFrame(columns=["lake", "dateTime", "meanRH850"])
    return pd.DataFrame(
        {
            "lake": [grid["lake"].iloc[0]],
            "dateTime" : [grid["timestamp"].iloc[0]],
            "meanRH850" : [grid["rh850"].mean()],
        }
    )

In [9]:
print(avgRH850("Cayuga Lake", "2026-1-1 0:00"))

Saved Cayuga Lake: 1,020,753 bytes → finger_lakes_geojson/cayuga_lake.geojson
✅ Found ┊ model=hrrr ┊ product=prs ┊ 2026-Jan-01 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


/opt/miniconda3/envs/herbie-hrrr/lib/python3.11/site-packages/herbie/core.py:1280: UserWarning: Will not remove GRIB file because it previously existed.
  warnings.warn("Will not remove GRIB file because it previously existed.")


          lake   dateTime  meanRH850
0  Cayuga Lake 2026-01-01  81.903435
